# VencoPy Tutorial 2

This tutorial aims to give a more in depth overview into the dataParser class and showcases some features that can be customised.

In [5]:
import sys
import os
from os import path
import pandas as pd
from pathlib import Path
import pathlib
#from ruamel.yaml import YAML

sys.path.append(path.dirname(path.dirname(path.dirname(Path.cwd()))))

from vencopy.classes.dataParsers import DataParser
from vencopy.scripts.globalFunctions import loadConfigDict

print("Current working directory: {0}".format(os.getcwd()))

Current working directory: C:\8_Work\VencoPy\vencopy_internal\vencopy\vencopy\tutorials\tutorial_02


In [6]:
configNames = ('globalConfig', 'localPathConfig', 'parseConfig', 'tripConfig', 'gridConfig', 'flexConfig', 'evaluatorConfig')
configDict = loadConfigDict(configNames)
    
# Set reference dataset 
datasetID = 'MiD17'

# Modify the localPathConfig file to point to the .csv file in the sampling folder in the tutorials directory where the dataset for the tutorials lies.
configDict['localPathConfig']['pathAbsolute'][datasetID] = Path.cwd().parent / 'data_sampling'

# Assign to vencoPyRoot the folder in which you cloned your repository
#localPathConfig['pathAbsolute']['vencoPyRoot'] = Path.cwd().parent.parent

# Similarly we modify the datasetID in the global config file
configDict['globalConfig']['files'][datasetID]['tripsDataRaw'] = datasetID + '.csv'

# Adapt relative paths in config for tutorials
configDict['globalConfig']['pathRelative']['plots'] = Path.cwd().parent.parent / configDict['globalConfig']['pathRelative']['plots']
configDict['globalConfig']['pathRelative']['parseOutput'] = Path.cwd().parent.parent / configDict['globalConfig']['pathRelative']['parseOutput']
configDict['globalConfig']['pathRelative']['diaryOutput'] = Path.cwd().parent.parent / configDict['globalConfig']['pathRelative']['diaryOutput']
configDict['globalConfig']['pathRelative']['gridOutput'] = Path.cwd().parent.parent / configDict['globalConfig']['pathRelative']['gridOutput']
configDict['globalConfig']['pathRelative']['flexOutput'] = Path.cwd().parent.parent / configDict['globalConfig']['pathRelative']['flexOutput']
configDict['globalConfig']['pathRelative']['evalOutput'] = Path.cwd().parent.parent / configDict['globalConfig']['pathRelative']['evalOutput']

# We also modify the parseConfig by removing some of the columns that are normally parsed from the MiD, which are not available in our semplified test dataframe
del configDict['parseConfig']['dataVariables']['hhID']
del configDict['parseConfig']['dataVariables']['personID']


## DataParser config file

The DataParser config file defines which variables are to be parsed (i.e. the ones needed to create trip diaries and calculate fleet flexibility) and sets some filtering options, such as the conditions for trips to be included of excluded from the parsing.

<div class="alert alert-block alert-danger"><b>Warning:</b> The list is very long.</div>

In [7]:
#yaml.dump(configDict['parseConfig'], sys.stdout)

## _DataParser_ class

Let's first run the class and see the outputs we get.

In [8]:
vpData = DataParser(datasetID=datasetID, configDict=configDict, loadEncrypted=False)
vpData.process()
vpData.data.head()

Parsing properties set up
Starting to retrieve local data file from C:\8_Work\VencoPy\vencopy_internal\vencopy\vencopy\tutorials\data_sampling\MiD17.csv
Finished loading 2124 rows of raw data of type .csv
Finished harmonization of variables
Starting filtering, applying 8 filters.
The following values were taken into account after filtering:
{'isMIVDriver': 1287,
 'tripDistance': 1948,
 'tripEndClock': 2124,
 'tripEndHour': 2124,
 'tripIsIntermodal': 1682,
 'tripPurpose': 2115,
 'tripStartClock': 2124,
 'tripStartHour': 2124}
All filters combined yielded a total of 950 was taken into account
This corresponds to 44.72693032015066 percent of the original data
Finished harmonization of ID variables
Parsing completed


,isMIVDriver,hhPersonID,tripID,tripWeight,tripScaleFactor,tripStartClock,tripEndClock,tripPurpose,tripDistance,travelTime,...,tripStartMinute,tripEndHour,tripEndMinute,tripEndNextDay,tripIsIntermodal,weekdayStr,purposeStr,timestampStart,timestampEnd,genericID
3,True,1410,4,0.146627,39.299224,50400.0,53400.0,2,9.50,50.0,...,0,14,50,False,0,MON,WORK,2017-03-13 14:00:00,2017-03-13 14:50:00,1410
19,True,636,1,0.190006,50.925648,22200.0,23400.0,1,13.30,20.0,...,10,6,30,False,0,TUE,WORK,2016-07-24 06:10:00,2016-07-24 06:30:00,636
20,True,636,2,0.190006,50.925648,64800.0,66000.0,8,13.30,20.0,...,0,18,20,False,0,TUE,HOME,2016-07-24 18:00:00,2016-07-24 18:20:00,636
32,True,87,2,0.359856,96.449272,39900.0,40500.0,5,2.38,10.0,...,5,11,15,False,0,WED,SHOPPING,2017-05-17 11:05:00,2017-05-17 11:15:00,87
33,True,87,3,0.359856,96.449272,40500.0,41100.0,8,2.38,10.0,...,15,11,25,False,0,WED,HOME,2017-05-17 11:15:00,2017-05-17 11:25:00,87


We can see from the print statements in the class that after reading in the initial dataset, which contained 2124 rows, and applying 8 filters, we end up with a database containing 950 suitable entries, which corresponds to about 45% of the initial sample.
These trip respect the condition that they all need to be shorter than 1000km, which is set in the parseConfig under the 'filterDict' key.

Now we can, for example, change in the filters the maximum allowed trip distance from 1000km to 50km and see how this affects the resulting available trips (the extreme case of 50km is only used for the tutorial purpose).

In [10]:
configDict['parseConfig']['filterDicts']['MiD17']['smallerThan']['tripDistance'] = [50]
#yaml.dump(configDict['parseConfig'], sys.stdout)

In [11]:
vpData = DataParser(datasetID=datasetID, configDict=configDict, loadEncrypted=False)
vpData.process()
vpData.data.head()

Parsing properties set up
Starting to retrieve local data file from C:\8_Work\VencoPy\vencopy_internal\vencopy\vencopy\tutorials\data_sampling\MiD17.csv
Finished loading 2124 rows of raw data of type .csv
Finished harmonization of variables
Starting filtering, applying 8 filters.
The following values were taken into account after filtering:
{'isMIVDriver': 1287,
 'tripDistance': 1892,
 'tripEndClock': 2124,
 'tripEndHour': 2124,
 'tripIsIntermodal': 1682,
 'tripPurpose': 2115,
 'tripStartClock': 2124,
 'tripStartHour': 2124}
All filters combined yielded a total of 914 was taken into account
This corresponds to 43.03201506591337 percent of the original data
Finished harmonization of ID variables
Parsing completed


,isMIVDriver,hhPersonID,tripID,tripWeight,tripScaleFactor,tripStartClock,tripEndClock,tripPurpose,tripDistance,travelTime,...,tripStartMinute,tripEndHour,tripEndMinute,tripEndNextDay,tripIsIntermodal,weekdayStr,purposeStr,timestampStart,timestampEnd,genericID
3,True,1410,4,0.146627,39.299224,50400.0,53400.0,2,9.50,50.0,...,0,14,50,False,0,MON,WORK,2017-03-13 14:00:00,2017-03-13 14:50:00,1410
19,True,636,1,0.190006,50.925648,22200.0,23400.0,1,13.30,20.0,...,10,6,30,False,0,TUE,WORK,2016-07-24 06:10:00,2016-07-24 06:30:00,636
20,True,636,2,0.190006,50.925648,64800.0,66000.0,8,13.30,20.0,...,0,18,20,False,0,TUE,HOME,2016-07-24 18:00:00,2016-07-24 18:20:00,636
32,True,87,2,0.359856,96.449272,39900.0,40500.0,5,2.38,10.0,...,5,11,15,False,0,WED,SHOPPING,2017-05-17 11:05:00,2017-05-17 11:15:00,87
33,True,87,3,0.359856,96.449272,40500.0,41100.0,8,2.38,10.0,...,15,11,25,False,0,WED,HOME,2017-05-17 11:15:00,2017-05-17 11:25:00,87


We can see how with a maximum trip distance of 1000km, all filters combined yielded a total of 950 trips, which corresponds to about 45% of the original dataset. By changing this values to 50km, additional 36 trips have been excluded, resulting in 914 trips (43% ofthe initial dataset).

## Next Steps

In the next tutorial, you will learn more in detail the internal workings of the TripDiaryBuilder class and how to customise some settings.